In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Given constants
T = 0.5              # 6 months
sigma = 0.3
r = 0.03
K = 1                # Strike set to 1 without loss of generality

# Theta function in terms of x = S/K
def theta(x):
    S = x * K
    d1 = (np.log(x) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    term1 = - (sigma * S) / (2 * np.sqrt(2 * np.pi * T)) * np.exp(-0.5 * d1**2)
    term2 = r * K * np.exp(-r * T) * norm.cdf(-d2)
    return term1 + term2

# Derivative of theta with respect to x
def dtheta_dx(x):
    S = x * K
    d1 = (np.log(x) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    phi_d1 = np.exp(-0.5 * d1**2) / np.sqrt(2 * np.pi)
    phi_d2 = np.exp(-0.5 * d2**2) / np.sqrt(2 * np.pi)

    d_d1_dx = 1 / (x * sigma * np.sqrt(T))
    d_d2_dx = d_d1_dx

    term1 = - (sigma * K) / (2 * np.sqrt(2 * np.pi * T)) * np.exp(-0.5 * d1**2)
    term2 = - (sigma * K * x) / (2 * np.sqrt(2 * np.pi * T)) * np.exp(-0.5 * d1**2) * (-d1) * d_d1_dx
    term3 = r * K * np.exp(-r * T) * (-phi_d2 * d_d2_dx)
    return term1 + term2 + term3

# General Newton method
def newton_method(f, df, x0, tol=1e-8, max_iter=2000):
    data = []
    x = x0
    for i in range(max_iter):
        fx = f(x)
        dfx = df(x)
        if dfx == 0:
            break
        x_new = x - fx / dfx
        error = abs(x_new - x)
        data.append({
            "iteration": i,
            "x": x,
            "f(x)": fx,
            "f'(x)": dfx,
            "x_new": x_new,
            "error": error
        })
        if error < tol:
            break
        x = x_new
    return pd.DataFrame(data)

df = newton_method(theta, dtheta_dx, x0=0.7)


In [8]:
print(df.to_latex(caption="Newton's Method for Theta Function", label="tab:newton_theta", index=False, escape=True))

\begin{table}
\caption{Newton's Method for Theta Function}
\label{tab:newton_theta}
\begin{tabular}{rrrrrr}
\toprule
iteration & x & f(x) & f'(x) & x\_new & error \\
\midrule
0 & 0.700000 & 0.009182 & -0.239005 & 0.738419 & 0.038419 \\
1 & 0.738419 & -0.001072 & -0.292386 & 0.734752 & 0.003667 \\
2 & 0.734752 & -0.000008 & -0.287984 & 0.734724 & 0.000028 \\
3 & 0.734724 & -0.000000 & -0.287950 & 0.734724 & 0.000000 \\
\bottomrule
\end{tabular}
\end{table}

